In [ ]:
from .policies.surv.algorithms.ADD4.polygon_grid_projector import PolygonGridProjector

mission_area_latlon = [
    [36.49469109, 126.5191631],
    [36.49469109, 126.58190534],
    [36.4503802, 126.58190534],
    [36.4503802, 126.5191631],
]
mission_capable_area_latlon = [
    [36.4761968, 126.5420048],
    [36.46494623, 126.52981684],
    [36.45334886, 126.53153345],
    [36.45252044, 126.55899927],
    [36.46894968, 126.56655237],
]

grid = (50, 50)
top_left, top_right, bottom_right, bottom_left = mission_area_latlon
in_pos = [2450, 2404]
pgp = PolygonGridProjector(grid_size=grid[0])
obs_pos, _, _ = pgp.obs_pos_from_polygon_lonlat(
    [top_left, bottom_left, top_right, bottom_right],
    mission_capable_area_latlon,
    None,
    use_hull=False,
)
obs_pos

In [ ]:
from shapely.geometry import Polygon

shell = [
    [126.5191631, 36.49469109],
    [126.58190534, 36.49469109],
    [126.58190534, 36.4503802],
    [126.5191631, 36.4503802],
]
holes = [
    [126.5420048, 36.4761968],
    [126.52981684, 36.46494623],
    [126.53153345, 36.45334886],
    [126.55899927, 36.45252044],
    [126.56655237, 36.46894968],
]

Polygon(shell=shell, holes=[holes])

In [351]:
import numpy as np
from PIL import Image, ImageDraw

mission_area_latlon = [
    [36.49469109, 126.5191631],
    [36.49469109, 126.58190534],
    [36.4503802, 126.58190534],
    [36.4503802, 126.5191631],
]
mission_capable_area_latlon = [
    [36.4761968, 126.5420048],
    [36.46494623, 126.52981684],
    [36.45334886, 126.53153345],
    [36.45252044, 126.55899927],
    [36.46894968, 126.56655237],
]
use_hull = False


def _cross(o, a, b):
    return (a[0] - o[0]) * (b[1] - o[1]) - (a[1] - o[1]) * (b[0] - o[0])


def _convex_hull_xy(self, points_xy):
    pts = sorted(set(map(tuple, points_xy)))
    if len(pts) <= 1:
        return pts
    lower = []
    for p in pts:
        while len(lower) >= 2 and _cross(lower[-2], lower[-1], p) <= 0:
            lower.pop()
        lower.append(p)
    upper = []
    for p in reversed(pts):
        while len(upper) >= 2 and _cross(upper[-2], upper[-1], p) <= 0:
            upper.pop()
        upper.append(p)
    return lower[:-1] + upper[:-1]


lats = [p[0] for p in mission_area_latlon]
lons = [p[1] for p in mission_area_latlon]
min_lat, max_lat = min(lats), max(lats)
min_lon, max_lon = min(lons), max(lons)

# polygon (x=lon, y=lat)
poly_xy = [(p[1], p[0]) for p in mission_capable_area_latlon]
used_xy = _convex_hull_xy(poly_xy) if use_hull else poly_xy

# to pixel (y-down)
W = H = 50
grid = np.asarray(used_xy, dtype=float)
px = (grid[:, 0] - min_lon) / (max_lon - min_lon) * (W - 1)
py = (1.0 - (grid[:, 1] - min_lat) / (max_lat - min_lat)) * (H - 1)
poly_pixels = list(map(tuple, np.stack([px, py], axis=1)))

# rasterize → mask (폴리곤 내부 = 1 = free space, 외부 = 0 = obstacle)
img = Image.new("L", (W, H), 0)
ImageDraw.Draw(img).polygon(poly_pixels, fill=1, outline=1)
mask = np.array(img, dtype=np.uint8)


grid_coverage_data = downsample_grid_converage_data(grid_coverage_data)

grid = 1 - grid_coverage_data.astype(np.uint8)

grid[(grid != 0) & (mask != 1)] = 0

grid = np.rot90(grid, k=-1)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(grid)